<a href="https://colab.research.google.com/github/preferencial/app/blob/main/io.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Instalação de Pacotes
!pip install pandas numpy matplotlib seaborn requests

# Importações
import os
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from google.colab import drive

# Configurações de Estilo
FIGSIZE = (12, 8)
sns.set(style="darkgrid")
plt.rcParams.update({
    'figure.facecolor': 'black', 'axes.facecolor': 'black', 'axes.edgecolor': 'white',
    'axes.labelcolor': 'cyan', 'xtick.color': 'white', 'ytick.color': 'white',
    'text.color': 'white', 'grid.color': 'gray', 'grid.linestyle': '--',
    'legend.facecolor': 'black', 'legend.edgecolor': 'white', 'figure.titlesize': 20,
    'axes.titlesize': 16, 'axes.labelsize': 14, 'xtick.labelsize': 12, 'ytick.labelsize': 12,
    'legend.fontsize': 12,
})

# Funções Auxiliares
def mount_drive():
    drive.mount('/content/drive')

def get_drive_path(relative_path):
    return os.path.join('/content/drive/MyDrive', relative_path)

def ensure_directory(path):
    os.makedirs(path, exist_ok=True)
    return path

def save_plot(fig, filename, folder):
    filepath = os.path.join(folder, filename)
    fig.savefig(filepath, bbox_inches='tight', dpi=300)
    plt.close(fig)
    print(f"Gráfico salvo: {filepath}")

def save_summary(summary_text, filename, folder):
    filepath = os.path.join(folder, filename)
    with open(filepath, 'w') as f:
        f.write(summary_text)
    print(f"Sumário salvo: {filepath}")


def plot_categorical_count(df, column, title, filename, folder, hue=None, third_variable=None, limit=6):
    """Plota a contagem de categorias, com hue, terceira variável e limite."""
    if column not in df.columns:
        print(f"Coluna '{column}' não encontrada.")
        return

    fig, ax = plt.subplots(figsize=FIGSIZE)

    value_counts = df[column].value_counts()
    top_categories = value_counts.head(limit)
    other_count = value_counts.iloc[limit:].sum()
    if other_count > 0:
        top_categories['Outros'] = other_count
    data = df[df[column].isin(top_categories.index)]


    if third_variable and hue:
        palette = sns.color_palette("coolwarm", n_colors=data[third_variable].nunique())
        sns.countplot(y=column, data=data, hue=hue, palette=palette, ax=ax, order=top_categories.index)
    elif hue:
        sns.countplot(y=column, data=data, hue=hue, palette="coolwarm", ax=ax, order=top_categories.index)
    else:
        sns.countplot(y=column, data=data, palette=["#FF5733", "#3498DB"], ax=ax, order=top_categories.index)

    ax.set_title(title)
    ax.set_xlabel("Contagem")
    ax.set_ylabel(column.replace('_', ' '))
    if hue:
        ax.legend(title=hue.replace('_', ' '), loc='lower right')
    save_plot(fig, filename, folder)


def plot_numeric_histogram(df, column, title, filename, folder, hue=None, limit=6):
    """Histograma com KDE, hue e limitando categorias do hue."""
    if column not in df.columns or not pd.api.types.is_numeric_dtype(df[column]):
        print(f"Coluna '{column}' não encontrada ou não numérica.")
        return

    fig, ax = plt.subplots(figsize=FIGSIZE)

    if hue:
        # Limitando categorias do hue
        top_hues = df[hue].value_counts().head(limit).index
        df_filtered = df[df[hue].isin(top_hues)]

        for cat in df_filtered[hue].unique():
            sns.kdeplot(df_filtered[df_filtered[hue] == cat][column], ax=ax, label=cat, fill=True)
        ax.legend(title=hue.replace('_', ' '))
    else:
        sns.histplot(df[column], kde=True, ax=ax, color="#FF5733")

    ax.set_title(title)
    ax.set_xlabel(column.replace('_', ' '))
    ax.set_ylabel("Densidade")
    save_plot(fig, filename, folder)


def plot_scatter(df, x_col, y_col, title, filename, folder, hue=None, third_variable=None, limit=6):
    """Gráfico de dispersão com hue, terceira variável e limite de categorias."""
    if not all(col in df.columns for col in [x_col, y_col]):
        print("Colunas necessárias não encontradas.")
        return
    # Modificação aqui: Permitir colunas categóricas se hue estiver definido
    if not (pd.api.types.is_numeric_dtype(df[x_col]) and pd.api.types.is_numeric_dtype(df[y_col])) and hue is None:
        print("Colunas x e y devem ser numéricas se não houver hue.")
        return

    fig, ax = plt.subplots(figsize=FIGSIZE)

    if hue:
        top_hues = df[hue].value_counts().head(limit).index
        df_filtered = df[df[hue].isin(top_hues)]
    else:
        df_filtered = df

    if third_variable:
        top_third = df_filtered[third_variable].value_counts().head(limit).index
        df_filtered = df_filtered[df_filtered[third_variable].isin(top_third)]
        palette = sns.color_palette("coolwarm", n_colors=df_filtered[third_variable].nunique())
        sns.scatterplot(data=df_filtered, x=x_col, y=y_col, hue=hue, palette=palette, size=third_variable, ax=ax)

    elif hue:
        sns.scatterplot(data=df_filtered, x=x_col, y=y_col, hue=hue, palette="coolwarm", ax=ax)
    else:
        sns.scatterplot(data=df_filtered, x=x_col, y=y_col, color="#FF5733", ax=ax)

    ax.set_title(title)
    ax.set_xlabel(x_col.replace('_', ' '))
    ax.set_ylabel(y_col.replace('_', ' '))
    save_plot(fig, filename, folder)



def plot_correlation_heatmap(df, title, filename, folder):
    """Plota um heatmap de correlação."""
    numeric_df = df.select_dtypes(include=[np.number])
    if numeric_df.empty:
        print("Sem colunas numéricas para o heatmap.")
        return

    fig, ax = plt.subplots(figsize=FIGSIZE)
    sns.heatmap(numeric_df.corr(), annot=True, cmap="coolwarm", fmt=".2f", ax=ax, linewidths=.5)
    ax.set_title(title)
    save_plot(fig, filename, folder)


def plot_boxplot(df, x_col, y_col, title, filename, folder, hue=None, third_variable=None, limit=6):
    """Boxplot com hue, terceira variável e limite de categorias."""
    if not all(col in df.columns for col in [x_col, y_col]):
        print("Colunas necessárias não encontradas.")
        return
    if not pd.api.types.is_numeric_dtype(df[y_col]):
        print("A coluna y deve ser numérica.")
        return

    fig, ax = plt.subplots(figsize=FIGSIZE)

    # Limitando categorias
    if hue:
        top_hues = df[hue].value_counts().head(limit).index
        df_filtered = df[df[hue].isin(top_hues)]
    else:
        df_filtered = df

    top_x = df_filtered[x_col].value_counts().head(limit).index
    df_filtered = df_filtered[df_filtered[x_col].isin(top_x)]

    if third_variable:
        top_third = df_filtered[third_variable].value_counts().head(limit).index
        df_filtered = df_filtered[df_filtered[third_variable].isin(top_third)]
        palette = sns.color_palette("coolwarm", n_colors=df_filtered[third_variable].nunique())
        sns.boxplot(x=x_col, y=y_col, data=df_filtered, hue=hue, palette=palette, ax=ax)

    elif hue:
        sns.boxplot(x=x_col, y=y_col, data=df_filtered, hue=hue, palette="coolwarm", ax=ax)
    else:
        sns.boxplot(x=x_col, y=y_col, data=df_filtered, palette=["#FF5733", "#3498DB"], ax=ax)

    ax.set_title(title)
    ax.set_xlabel(x_col.replace('_', ' '))
    ax.set_ylabel(y_col.replace('_', ' '))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    save_plot(fig, filename, folder)


def plot_violin(df, x_col, y_col, title, filename, folder, hue=None, third_variable=None, limit=6):
    """Violin plot com hue, terceira variável e limite de categorias."""
    if not all(col in df.columns for col in [x_col, y_col]):
        print("Colunas necessárias não encontradas.")
        return
    if not pd.api.types.is_numeric_dtype(df[y_col]):
        print("A coluna y deve ser numérica.")
        return

    fig, ax = plt.subplots(figsize=FIGSIZE)

    # Limitando categorias
    if hue:
        top_hues = df[hue].value_counts().head(limit).index
        df_filtered = df[df[hue].isin(top_hues)]
    else:
        df_filtered = df

    top_x = df_filtered[x_col].value_counts().head(limit).index
    df_filtered = df_filtered[df_filtered[x_col].isin(top_x)]


    if third_variable:
        top_third = df_filtered[third_variable].value_counts().head(limit).index
        df_filtered = df_filtered[df_filtered[third_variable].isin(top_third)]
        palette = sns.color_palette("coolwarm", n_colors=df_filtered[third_variable].nunique())
        sns.violinplot(x=x_col, y=y_col, data=df_filtered, hue=hue, palette=palette, ax=ax, split=(hue is not None))

    elif hue:
        sns.violinplot(x=x_col, y=y_col, data=df_filtered, hue=hue, palette="coolwarm", ax=ax, split=True)
    else:
        sns.violinplot(x=x_col, y=y_col, data=df_filtered, palette=["#FF5733", "#3498DB"], ax=ax)

    ax.set_title(title)
    ax.set_xlabel(x_col.replace('_', ' '))
    ax.set_ylabel(y_col.replace('_', ' '))
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    save_plot(fig, filename, folder)


def plot_line(df, x_col, y_col, title, filename, folder, hue=None, third_variable=None, limit=6):
    """Gráfico de linha com hue, terceira variável e limite de categorias."""
    if not all(col in df.columns for col in [x_col, y_col]):
        print("Colunas necessárias não encontradas.")
        return
    if not pd.api.types.is_numeric_dtype(df[y_col]):
        print("A coluna y deve ser numérica.")
        return

    fig, ax = plt.subplots(figsize=FIGSIZE)

    if hue:
        top_hues = df[hue].value_counts().head(limit).index
        df_filtered = df[df[hue].isin(top_hues)]
    else:
        df_filtered = df

    if third_variable:
        top_third = df_filtered[third_variable].value_counts().head(limit).index
        df_filtered = df_filtered[df_filtered[third_variable].isin(top_third)]
        palette = sns.color_palette("coolwarm", n_colors=df_filtered[third_variable].nunique())
        sns.lineplot(x=x_col, y=y_col, data=df_filtered, hue=hue, palette=palette, size=third_variable, ax=ax)

    elif hue:
        sns.lineplot(x=x_col, y=y_col, data=df_filtered, hue=hue, palette="coolwarm", ax=ax)
    else:
        sns.lineplot(x=x_col, y=y_col, data=df_filtered, color="#FF5733", ax=ax)

    ax.set_title(title)
    ax.set_xlabel(x_col.replace('_', ' '))
    ax.set_ylabel(y_col.replace('_', ' '))
    save_plot(fig, filename, folder)



# Função Principal
def main():
    try:
        mount_drive()
        data_folder = ensure_directory(get_drive_path('data'))
        url = "https://docs.google.com/spreadsheets/d/1bSdIlZ3y0rCbhBtvw77P1JdxTYx6AfFYM-yWvIuP3nk/export?format=csv"
        response = requests.get(url)
        response.raise_for_status()
        df = pd.read_csv(StringIO(response.text))

        # Pré-processamento
        df.columns = df.columns.str.replace(' ', '_')
        df.dropna(axis=1, how='all', inplace=True)
        for col in df.select_dtypes(include=['object']).columns:
            df[col] = df[col].str.strip()
        df['UltimaAtualizacao'] = pd.to_datetime(df['UltimaAtualizacao'], errors='coerce')

        # TRATAMENTO DE DADOS CATEGORICOS PARA NUMERICOS
        # 1. FrequenciaUso:
        frequencia_map = {'Diaria': 1, 'Semanal': 2, 'Mensal': 3, 'Raramente': 4, 'Nunca': 5}
        df['FrequenciaUso'] = df['FrequenciaUso'].map(frequencia_map)

        # --- Geração de Gráficos e Sumário ---
        summary = ""

        # 1. Contagem de Pacotes por Categoria Principal
        plot_categorical_count(df, 'CategoriaPrincipal', "Pacotes por Categoria", "pacotes_por_categoria.png", data_folder, hue='InstalacaoViaPip', third_variable='NivelSatisfacao')
        summary += "1. Contagem de Pacotes por Categoria Principal:\n" + df['CategoriaPrincipal'].value_counts().head(6).to_string() + "\n\n"

        # 2. Distribuição da Frequência de Uso
        plot_categorical_count(df, 'FrequenciaUso', "Distribuição da Frequência de Uso", "distribuicao_frequencia_uso.png", data_folder, hue='CategoriaPrincipal', third_variable='NivelSatisfacao')
        summary += "2. Distribuição da Frequência de Uso:\n" + df['FrequenciaUso'].value_counts().head(6).to_string() + "\n\n"

        # 3. Histograma do Nível de Satisfação
        plot_numeric_histogram(df, 'NivelSatisfacao', "Distribuição do Nível de Satisfação", "distribuicao_nivel_satisfacao.png", data_folder, hue='FrequenciaUso')
        summary += "3. Distribuição do Nível de Satisfação:\n" + df['NivelSatisfacao'].describe().to_string() + "\n\n"

        # 4. Scatter Plot: Frequência de Uso vs. Nível de Satisfação
        plot_scatter(df, 'FrequenciaUso', 'NivelSatisfacao', "Frequência de Uso vs. Nível de Satisfação", "frequencia_vs_satisfacao.png", data_folder, hue='CategoriaPrincipal', third_variable='InstalacaoViaPip')
        summary += "4. Scatter Plot: Frequência de Uso vs. Nível de Satisfação:\nCorrelação:\n" + df[['FrequenciaUso', 'NivelSatisfacao']].corr().to_string() + "\n\n"

        # 5. Boxplot: Nível de Satisfação por Categoria Principal
        plot_boxplot(df, 'CategoriaPrincipal', 'NivelSatisfacao', "Nível de Satisfação por Categoria", "satisfacao_por_categoria_boxplot.png", data_folder, hue='InstalacaoViaPip')
        summary += "6. Boxplot: Nível de Satisfação por Categoria Principal:\nEstatísticas descritivas por categoria (Top 6):\n" + df.groupby('CategoriaPrincipal')['NivelSatisfacao'].describe().head(6).to_string() + "\n\n"

        # 6. Violin Plot: Nível de Satisfação por Frequência de Uso
        plot_violin(df, 'FrequenciaUso', 'NivelSatisfacao', "Nível de Satisfação por Frequência de Uso", "satisfacao_por_frequencia_violin.png", data_folder, hue='InstalacaoViaPip')
        summary += "7. Violin Plot: Nível de Satisfação por Frequência de Uso:\nEstatísticas descritivas por frequência (Top 6):\n" + df.groupby('FrequenciaUso')['NivelSatisfacao'].describe().head(6).to_string() + "\n\n"

        # 7. Contagem de Instalação via Pip
        plot_categorical_count(df, 'InstalacaoViaPip', "Contagem de Instalação via Pip", "instalacao_via_pip.png", data_folder, hue='CategoriaPrincipal')
        summary += "8. Contagem de Instalação via Pip:\n" + df['InstalacaoViaPip'].value_counts().head(6).to_string() + "\n\n"

        save_summary(summary, "sumario.txt", data_folder)
        print("Análise completa, gráficos e sumário gerados!")

    except requests.exceptions.RequestException as e:
        print(f"Erro ao acessar o URL: {e}")
    except Exception as e:
        print(f"Erro geral: {e}")

if __name__ == "__main__":
    main()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-12-a9cb2a5d2915>:68: UserWarning: The palette list has more values (4) than needed (1), which may not be intended.
  sns.countplot(y=column, data=data, hue=hue, palette=palette, ax=ax, order=top_categories.index)


Gráfico salvo: /content/drive/MyDrive/data/pacotes_por_categoria.png


<ipython-input-12-a9cb2a5d2915>:68: UserWarning: 
The palette list has fewer values (4) than needed (23) and will cycle, which may produce an uninterpretable plot.
  sns.countplot(y=column, data=data, hue=hue, palette=palette, ax=ax, order=top_categories.index)


Gráfico salvo: /content/drive/MyDrive/data/distribuicao_frequencia_uso.png
Gráfico salvo: /content/drive/MyDrive/data/distribuicao_nivel_satisfacao.png


...

Gráfico salvo: /content/drive/MyDrive/data/satisfacao_por_frequencia_violin.png
Gráfico salvo: /content/drive/MyDrive/data/instalacao_via_pip.png
Gráfico salvo: /content/drive/MyDrive/data/evolucao_versao_pandas.png


<ipython-input-12-a9cb2a5d2915>:191: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")


Gráfico salvo: /content/drive/MyDrive/data/satisfacao_por_instalacao.png


<ipython-input-12-a9cb2a5d2915>:231: UserWarning: set_ticklabels() should only be used with a fixed number of ticks, i.e. after set_ticks() or using a FixedLocator.
  ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")


Gráfico salvo: /content/drive/MyDrive/data/satisfacao_por_pacote_violin.png
Sumário salvo: /content/drive/MyDrive/data/sumario.txt
Análise completa, gráficos e sumário gerados!
